# Questions for Class Name

In [ ]:
import json
import pickle
import random

wiki = pickle.load(
    open(
        "ImageNet-Wiki_dataset/class_article_text_descriptions/new_class_article_text_descriptions_trainval.pkl",
        "rb",
    )
)

for key in wiki:
    output_str = f"{'='*63}\n\n"
    articles = wiki[key]["articles"]
    for idx, article in enumerate(articles):
        output_str += f"Article {idx+1}: "
        output_str += article.strip()
        output_str += f"\n\n{'='*63}\n\n"
    wiki[key]["concat_article"] = output_str.strip()

sampled_classes = list(wiki.keys())

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

client = OpenAI()


def fetch_class_questions(classname):
    prompt = f"""Come up with five multiple-choice questions for a {classname} to examine the knowledge of an expert.

The questions should come from the following Wikipedia articles on {classname}:
```
{wiki[classname]["concat_article"]}
```

**Instruction**:
Each question should have four choices, one of which is the correct answer. 
Note that the Wikipedia articles will not be accessible to the test-takers, so please do not reference specific details from the articles in the questions.
Use "this object" rather than "{classname}" in the questions. We want to give test-takers an image of this object, not the word itself.
Each question should have four fields: "question" (str), "choices" (list[str]), "answer" (int, starting from 0)), and "reference" (str, original sentences from Wikipedia articles).
Output in JSON format."""
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"},
    )
    output = json.loads(completion.choices[0].message.content)
    return classname, output


outputs = []
with ThreadPoolExecutor(max_workers=16) as executor:
    future_to_class = {
        executor.submit(fetch_class_questions, classname): classname
        for classname in sampled_classes
    }
    for future in tqdm(as_completed(future_to_class), total=len(sampled_classes)):
        classname, output = future.result()
        outputs.append((classname, output))

json.dump(outputs, open("classqa_wiki_all.json", "w"), indent=2)

# Rebalance Choices

In [ ]:
import json
import random

outputs = json.load(open("classqa_wiki_all.json"))
new_outputs = []
for classname, output in outputs:
    if "questions" not in output:
        output["questions"] = output["quiz_questions"]
        output.pop("quiz_questions")

random.seed(1234)
for classname, output in outputs:
    questions = output["questions"]
    for question in questions:
        choices = question["choices"]
        correct_answer = choices[question["answer"]]
        random.shuffle(choices)
        question["choices"] = choices
        question["answer"] = choices.index(correct_answer)

json.dump(outputs, open("classqa_wiki_all_balanced.json", "w"), indent=2)

# Verify Questions

In [ ]:
import json

outputs = json.load(open("classqa_wiki_all_balanced.json"))

questions = []

for classname, output in outputs:
    for question in output["questions"]:
        question["class"] = classname
        questions.append(question)

cnt = 0
for question in questions:
    if "this object" in question["question"]:
        question["question_with_classname"] = question["question"].replace(
            "this object", question["class"]
        )
    elif "This object" in question["question"]:
        question["question_with_classname"] = question["question"].replace(
            "This object", question["class"].capitalize()
        )
    elif "these objects" in question["question"]:
        question["question_with_classname"] = question["question"].replace(
            "these objects", question["class"] + "s"
        )
    else:
        cnt += 1
        continue

len(questions), questions[0], cnt

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

client = OpenAI()


def pred_question_with_classname(question):
    if "question_with_classname" not in question:
        return None
    prompt = f"""Answer the multiple-choice question below.
    {question["question_with_classname"]}
    A. {question["choices"][0]}
    B. {question["choices"][1]}
    C. {question["choices"][2]}
    D. {question["choices"][3]}
    Output only one character A/B/C/D."""
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}],
    )
    output = completion.choices[0].message.content
    try:
        pred = ["A", "B", "C", "D"].index(output[0])
    except:
        print(output)
        pred = output
    return pred


with ThreadPoolExecutor(max_workers=16) as executor:
    future_to_question = {
        executor.submit(pred_question_with_classname, question): question
        for question in questions
    }
    for future in tqdm(as_completed(future_to_question), total=len(questions)):
        question = future_to_question[future]
        pred = future.result()
        question["pred_with_classname"] = pred


json.dump(questions, open("classqa_wiki_all_balanced_answered.json", "w"), indent=2)

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

client = OpenAI()

questions = json.load(open("classqa_wiki_all_balanced_answered.json"))


def pred_question_without_classname(question):
    if "question_with_classname" not in question:
        return None
    prompt = f"""Answer the multiple-choice question below.
    {question["question"]}
    A. {question["choices"][0]}
    B. {question["choices"][1]}
    C. {question["choices"][2]}
    D. {question["choices"][3]}
    The question mentions "this object". However, we don't know what the object is. Try your best to guess the answer without knowing the object.
    Output only one character A/B/C/D."""
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}],
    )
    output = completion.choices[0].message.content
    try:
        pred = ["A", "B", "C", "D"].index(output[0])
    except:
        print(output)
        pred = output
    return pred


with ThreadPoolExecutor(max_workers=16) as executor:
    future_to_question = {
        executor.submit(pred_question_without_classname, question): question
        for question in questions
    }
    for future in tqdm(as_completed(future_to_question), total=len(questions)):
        question = future_to_question[future]
        pred = future.result()
        question["pred_without_classname"] = pred

json.dump(
    questions, open("classqa_wiki_all_balanced_answered_stage2.json", "w"), indent=2
)

In [ ]:
import json

questions = json.load(open("classqa_wiki_all_balanced_answered_stage2.json"))
questions = [
    question
    for question in questions
    if question["pred_with_classname"] is not None
    and question["pred_without_classname"] is not None
]

accuracy1 = sum(
    [question["pred_with_classname"] == question["answer"] for question in questions]
) / len(questions)
print("if we know classname", accuracy1)

accuracy2 = sum(
    [question["pred_without_classname"] == question["answer"] for question in questions]
) / len(questions)
print("if we don't know classname", accuracy2)

print(len(questions))

# Filter Questions

In [ ]:
import json

questions = json.load(open("classqa_wiki_all_balanced_answered_stage2.json"))

filtered_questions = []

for question in questions:
    if "question_with_classname" not in question:
        continue
    if (
        question["pred_without_classname"] != question["answer"]
        and question["pred_with_classname"] == question["answer"]
    ):
        filtered_questions.append(question)

print(len(filtered_questions), len(questions))

json.dump(
    filtered_questions, open("classqa_wiki_all_balanced_filtered.json", "w"), indent=2
)

# Rebalancing

In [ ]:
import random

questions = json.load(open("classqa_wiki_all_balanced_filtered.json"))

random.seed(1234)
for question in questions:
    choices = question["choices"]
    correct_answer = choices[question["answer"]]
    random.shuffle(choices)
    question["choices"] = choices
    question["answer"] = choices.index(correct_answer)

json.dump(
    questions, open("classqa_wiki_all_balanced_filtered_rebalanced.json", "w"), indent=2
)

# To LLaVA Questions

In [ ]:
import json

data = [json.loads(line) for line in open("../data/imagenet.jsonl")]

data = [item for item in data if item["split"] == "valid"]

label2images = {}
for item in data:
    label = item["label"]
    if label not in label2images:
        label2images[label] = []
    label2images[label].append(item["image"])

In [ ]:
import random

random.seed(1234)

multiple_choice_questions = json.load(
    open("classqa_wiki_all_balanced_filtered_rebalanced.json")
)

output_questions = []
for i, question in enumerate(multiple_choice_questions):
    classname = question["class"]
    images = random.sample(label2images[classname], 10)
    for j, image in enumerate(images):
        text = f"""{question['question']}\nA. {question['choices'][0]}\nB. {question['choices'][1]}\nC. {question['choices'][2]}\nD. {question['choices'][3]}\nChoose A, B, C, or D."""
        label = ["A", "B", "C", "D"][question["answer"]]
        output_question = {
            "image": image,
            "text": text,
            "category": "conv",
            "label": label,
            "question_id": f"{classname}_{label}_question{i}_image{j}",
        }
        output_questions.append(output_question)

random.shuffle(output_questions)
with open("imagenetqa_wiki_0517.jsonl", "w") as f:
    for output_question in output_questions:
        f.write(json.dumps(output_question) + "\n")

In [ ]:
import random

random.seed(1234)

multiple_choice_questions = json.load(
    open("classqa_wiki_all_balanced_filtered_rebalanced.json")
)

output_questions = []
for i, question in enumerate(multiple_choice_questions):
    classname = question["class"]
    images = random.sample(label2images[classname], 10)
    for j, image in enumerate(images):
        text = f"""{question['question_with_classname']}\nA. {question['choices'][0]}\nB. {question['choices'][1]}\nC. {question['choices'][2]}\nD. {question['choices'][3]}\nChoose A, B, C, or D."""
        label = ["A", "B", "C", "D"][question["answer"]]
        output_question = {
            "image": image,
            "text": text,
            "category": "conv",
            "label": label,
            "question_id": f"{classname}_{label}_question{i}_image{j}",
        }
        output_questions.append(output_question)

random.shuffle(output_questions)
with open("imagenetqa_wiki_withclassname_0517.jsonl", "w") as f:
    for output_question in output_questions:
        f.write(json.dumps(output_question) + "\n")